In [1]:
#Importing all necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import random
import json


2023-04-14 13:05:07.033730: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 13:05:07.088707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 13:05:08.497353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Tokenize the input and output pairs
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_pairs)
input_sequences = tokenizer.texts_to_sequences(input_pairs)
max_input_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences = tokenizer.texts_to_sequences(output_pairs)
max_output_length = max(len(seq) for seq in output_sequences)
output_sequences = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# Define the model
input_vocab_size = len(tokenizer.word_index) + 1
output_vocab_size = len(tokenizer.word_index) + 1
embedding_size = 128
units = 256

encoder_inputs = tf.keras.layers.Input(shape=(max_input_length,))
encoder_embedding = tf.keras.layers.Embedding(input_vocab_size, embedding_size)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(max_output_length,))
decoder_embedding = tf.keras.layers.Embedding(output_vocab_size, embedding_size)
decoder_lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([input_sequences, output_sequences[:,:-1]], output_sequences[:,1:], epochs=50, batch_size=64)